In [659]:
import pandas as pd
import openpyxl
from pathlib import Path

raw_national_df = pd.read_csv("Resources\Outpatient_Imaging_Efficiency-National.csv")
raw_state_df = pd.read_csv("Resources\Outpatient_Imaging_Efficiency-State.csv")

raw_df = pd.merge(raw_state_df, raw_national_df, how="left", on=["Measure ID", "Measure ID"])

raw_df.head()

,State,Measure ID,Measure Name_x,Score_x,Footnote_x,Start Date_x,End Date_x,Measure Name_y,Score_y,Footnote_y,Start Date_y,End Date_y
0,AK,OP_10,Abdomen CT Use of Contrast Material,5,NaN,07/01/2021,06/30/2022,Abdomen CT Use of Contrast Material,6.0,NaN,07/01/2021,06/30/2022
1,AK,OP_13,Outpatients who got cardiac imaging stress tes...,3.9,NaN,07/01/2021,06/30/2022,Outpatients who got cardiac imaging stress tes...,3.8,NaN,07/01/2021,06/30/2022
2,AK,OP_39,Breast Cancer Screening Recall Rates,7.5,NaN,07/01/2021,06/30/2022,Breast Cancer Screening Recall Rates,9.2,NaN,07/01/2021,06/30/2022
3,AK,OP_8,MRI Lumbar Spine for Low Back Pain,43.2,NaN,07/01/2021,06/30/2022,MRI Lumbar Spine for Low Back Pain,37.1,NaN,07/01/2021,06/30/2022
4,AL,OP_10,Abdomen CT Use of Contrast Material,9,NaN,07/01/2021,06/30/2022,Abdomen CT Use of Contrast Material,6.0,NaN,07/01/2021,06/30/2022


In [660]:
raw_df = raw_df[["State", "Measure ID", "Measure Name_x", "Score_x", "Score_y", "Start Date_x", "End Date_x"]]
raw_df.head()

,State,Measure ID,Measure Name_x,Score_x,Score_y,Start Date_x,End Date_x
0,AK,OP_10,Abdomen CT Use of Contrast Material,5,6.0,07/01/2021,06/30/2022
1,AK,OP_13,Outpatients who got cardiac imaging stress tes...,3.9,3.8,07/01/2021,06/30/2022
2,AK,OP_39,Breast Cancer Screening Recall Rates,7.5,9.2,07/01/2021,06/30/2022
3,AK,OP_8,MRI Lumbar Spine for Low Back Pain,43.2,37.1,07/01/2021,06/30/2022
4,AL,OP_10,Abdomen CT Use of Contrast Material,9,6.0,07/01/2021,06/30/2022


In [661]:
op_df = raw_df.rename(columns={"Measure ID": "Test Code", "Measure Name_x" : "Test Reason", "Score_x": "State Score",
                                "Score_y": "National Score", "Start Date_x": "Start Date", "End Date_x": "End Date"})
op_df.head()


,State,Test Code,Test Reason,State Score,National Score,Start Date,End Date
0,AK,OP_10,Abdomen CT Use of Contrast Material,5,6.0,07/01/2021,06/30/2022
1,AK,OP_13,Outpatients who got cardiac imaging stress tes...,3.9,3.8,07/01/2021,06/30/2022
2,AK,OP_39,Breast Cancer Screening Recall Rates,7.5,9.2,07/01/2021,06/30/2022
3,AK,OP_8,MRI Lumbar Spine for Low Back Pain,43.2,37.1,07/01/2021,06/30/2022
4,AL,OP_10,Abdomen CT Use of Contrast Material,9,6.0,07/01/2021,06/30/2022


In [662]:
cleaned_op_df = op_df[(op_df["State Score"] != "Not Available")][["State", "Test Code", "Test Reason", "State Score",
                                                                   "National Score", "Start Date", "End Date"]]
cleaned_op_df.head()

,State,Test Code,Test Reason,State Score,National Score,Start Date,End Date
0,AK,OP_10,Abdomen CT Use of Contrast Material,5,6.0,07/01/2021,06/30/2022
1,AK,OP_13,Outpatients who got cardiac imaging stress tes...,3.9,3.8,07/01/2021,06/30/2022
2,AK,OP_39,Breast Cancer Screening Recall Rates,7.5,9.2,07/01/2021,06/30/2022
3,AK,OP_8,MRI Lumbar Spine for Low Back Pain,43.2,37.1,07/01/2021,06/30/2022
4,AL,OP_10,Abdomen CT Use of Contrast Material,9,6.0,07/01/2021,06/30/2022


In [663]:
cleaned_op_df = cleaned_op_df.astype({"State Score": float}, errors='raise')
cleaned_op_df.dtypes

State              object
Test Code          object
Test Reason        object
State Score       float64
National Score    float64
Start Date         object
End Date           object
dtype: object

In [664]:
national_avg_df = cleaned_op_df.groupby("State")["National Score"].mean()
state_avg_df = cleaned_op_df.groupby("State")["State Score"].mean()
print(national_avg_df.head())
print(state_avg_df.head())

State
AK    14.025
AL    14.025
AR    14.025
AZ    14.025
CA    14.025
Name: National Score, dtype: float64
State
AK    14.900
AL    15.150
AR    14.700
AZ    13.175
CA    14.375
Name: State Score, dtype: float64


In [665]:
average_scores = pd.DataFrame({"State Average": state_avg_df, "National Average": national_avg_df})
average_scores.head()

,State Average,National Average
State,,
AK,14.900,14.025
AL,15.150,14.025
AR,14.700,14.025
AZ,13.175,14.025
CA,14.375,14.025


In [666]:
inefficent_states = average_scores[(average_scores["State Average"] > average_scores["National Average"])]
inefficent_states["Verdict"] = "Inefficient"
inefficent_states.head()

C:\Users\100676677\AppData\Local\Temp\ipykernel_17000\4051800990.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inefficent_states["Verdict"] = "Inefficient"


,State Average,National Average,Verdict
State,,,
AK,14.900,14.025,Inefficient
AL,15.150,14.025,Inefficient
AR,14.700,14.025,Inefficient
CA,14.375,14.025,Inefficient
CO,14.300,14.025,Inefficient


In [667]:
efficent_states = average_scores[(average_scores["State Average"] < average_scores["National Average"])]
efficent_states["Verdict"] = "Efficient"
efficent_states.head()

C:\Users\100676677\AppData\Local\Temp\ipykernel_17000\4210703357.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  efficent_states["Verdict"] = "Efficient"


,State Average,National Average,Verdict
State,,,
AZ,13.175,14.025,Efficient
DC,12.550,14.025,Efficient
DE,11.150,14.025,Efficient
GA,13.500,14.025,Efficient
HI,11.575,14.025,Efficient


In [668]:
verdict_df = pd.concat([inefficent_states, efficent_states])
verdict_df = verdict_df.sort_values(by = "State", ascending=True)
verdict_df

,State Average,National Average,Verdict
State,,,
AK,14.900,14.025,Inefficient
AL,15.150,14.025,Inefficient
AR,14.700,14.025,Inefficient
AZ,13.175,14.025,Efficient
CA,14.375,14.025,Inefficient
CO,14.300,14.025,Inefficient
CT,17.500,14.025,Inefficient
DC,12.550,14.025,Efficient
DE,11.150,14.025,Efficient
